In [1]:
import argparse
from collections import OrderedDict
import copy
from multiprocessing import Process,Manager
import numpy as np
import pandas as pd
from scipy import sparse
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter
import time
from tqdm import tqdm

import models
import data
import metric

In [2]:
import importlib
importlib.reload(metric)

<module 'metric' from '/home/jupyter/vae-cf-pytorch/metric.py'>

In [3]:
# Set Configs

In [4]:
##  Set the random seed manually for reproductibility.
seed = 1
torch.manual_seed(seed)

In [5]:
device= torch.device("cuda")
# device = torch.device("cpu")

In [6]:
# Load Data
loader = data.DataLoader('ml-20m')

n_items = loader.load_n_items()
train_data = loader.load_data('train')
vad_data_tr, vad_data_te = loader.load_data('validation')
test_data_tr, test_data_te = loader.load_data('test')

N = train_data.shape[0]
idxlist = list(range(N))

print("# of items:{}".format(n_items))

# of items:20101


In [7]:
# Build the model

p_dims = [200, 600, n_items]
model = models.MultiVAE(p_dims).to(device)

print(f"Model Structure:{model}\n")
# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.00)
criterion = models.loss_function

Model Structure:MultiVAE(
  (q_layers): ModuleList(
    (0): Linear(in_features=20101, out_features=600, bias=True)
    (1): Linear(in_features=600, out_features=400, bias=True)
  )
  (p_layers): ModuleList(
    (0): Linear(in_features=200, out_features=600, bias=True)
    (1): Linear(in_features=600, out_features=20101, bias=True)
  )
  (drop): Dropout(p=0.5, inplace=False)
)



In [8]:
# TensorboardX Writer
writer= SummaryWriter()

In [9]:
# Train

In [10]:
BATCH_SIZE = 500
TOTAL_ANNEAL_STEPS = 200000
ANNEAL_CAP = 0.2
LOG_INTERVAL = 100
# EPOCHS = 100
EPOCHS = 200
SAVE_PATH = 'model.pt'

In [11]:
def sparse2torch_sparse(data):
    """
    Convert scipy sparse matrix to torch sparse tensor with L2 Normalization
    This is much faster than naive use of torch.FloatTensor(data.toarray())
    https://discuss.pytorch.org/t/sparse-tensor-use-cases/22047/2
    """
    samples = data.shape[0]
    features = data.shape[1]
    coo_data = data.tocoo()
    indices = torch.LongTensor([coo_data.row, coo_data.col])
    row_norms_inv = 1 / np.sqrt(data.sum(1))
    row2val = {i : row_norms_inv[i].item() for i in range(samples)}
    values = np.array([row2val[r] for r in coo_data.row])
    t = torch.sparse.FloatTensor(indices, torch.from_numpy(values).float(), [samples, features])
    return t

In [12]:
def naive_sparse2tensor(data):
    return torch.FloatTensor(data.toarray())

In [13]:
def train():
    # Turn on training mode
    model.train()
    train_loss = 0.0
    start_time = time.time()
    global update_count

    np.random.shuffle(idxlist)
    
    for batch_idx, start_idx in enumerate(range(0, N, BATCH_SIZE)):
        end_idx = min(start_idx + BATCH_SIZE, N)
        data = train_data[idxlist[start_idx:end_idx]]
        data = naive_sparse2tensor(data).to(device)

        if TOTAL_ANNEAL_STEPS > 0:
            anneal = min(ANNEAL_CAP, 
                            1. * update_count / TOTAL_ANNEAL_STEPS)
        else:
            anneal = ANNEAL_CAP

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        
        loss = criterion(recon_batch, data, mu, logvar, anneal)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        update_count += 1

        if batch_idx % LOG_INTERVAL == 0 and batch_idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:4d}/{:4d} batches | ms/batch {:4.2f} | '
                    'loss {:4.2f}'.format(
                        epoch, batch_idx, len(range(0, N, BATCH_SIZE)),
                        elapsed * 1000 / LOG_INTERVAL,
                        train_loss / LOG_INTERVAL))
            
            # Log loss to tensorboard
            n_iter = (epoch - 1) * len(range(0, N, BATCH_SIZE)) + batch_idx
            writer.add_scalars('data/loss', {'train': train_loss / LOG_INTERVAL}, n_iter)

            start_time = time.time()
            train_loss = 0.0

In [14]:
def evaluate(data_tr, data_te):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    n1_list = []
    n100_list = []
    r20_list = []
    r50_list = []
    
    with torch.no_grad():
        for start_idx in range(0, e_N, BATCH_SIZE):
            end_idx = min(start_idx + BATCH_SIZE, N)
            data = data_tr[e_idxlist[start_idx:end_idx]]
            heldout_data = data_te[e_idxlist[start_idx:end_idx]]
    
            # cno : avoid users who have no clicks in heldout_data
            u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
            data = data[u_idxlist_wo_any_iteracts]
            heldout_data = heldout_data[u_idxlist_wo_any_iteracts]
            
            data_tensor = naive_sparse2tensor(data).to(device)

            if TOTAL_ANNEAL_STEPS > 0:
                anneal = min(ANNEAL_CAP, 
                               1. * update_count / TOTAL_ANNEAL_STEPS)
            else:
                anneal = ANNEAL_CAP

            recon_batch, mu, logvar = model(data_tensor)

            loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
            total_loss += loss.item()

            # Exclude examples from training set
            recon_batch = recon_batch.cpu().numpy()
            recon_batch[data.nonzero()] = -np.inf

            n1 = metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 1)
            n100 = metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100)
            r20 = metric.Recall_at_k_batch(recon_batch, heldout_data, 20)
            r50 = metric.Recall_at_k_batch(recon_batch, heldout_data, 50)

            n1_list.append(n1)
            n100_list.append(n100)
            r20_list.append(r20)
            r50_list.append(r50)
 
    total_loss /= len(range(0, e_N, BATCH_SIZE))
    n1_list = np.concatenate(n1_list)
    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)

    return total_loss, n1_list, n100_list, r20_list, r50_list

In [13]:
best_n100 = -np.inf
update_count = 0

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        train()
        val_loss, n100, r20, r50 = evaluate(vad_data_tr, vad_data_te)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:4.2f}s | valid loss {:4.2f} | '
                'n100 {:5.3f} | r20 {:5.3f} | r50 {:5.3f}'.format(
                    epoch, time.time() - epoch_start_time, val_loss,
                    n100, r20, r50))
        print('-' * 89)

        n_iter = epoch * len(range(0, N, BATCH_SIZE))
        writer.add_scalars('data/loss', {'valid': val_loss}, n_iter)
        writer.add_scalar('data/n100', n100, n_iter)
        writer.add_scalar('data/r20', r20, n_iter)
        writer.add_scalar('data/r50', r50, n_iter)

        # Save the model if the n100 is the best we've seen so far.
        if n100 > best_n100:
            with open(SAVE_PATH, 'wb') as f:
                torch.save(model, f)
            best_n100 = n100

except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

print(update_count)

| epoch   1 |  100/ 233 batches | ms/batch 134.92 | loss 572.41
-----------------------------------------------------------------------------------------
Exiting from training early


In [15]:
# Load the best saved model.
MODEL_PATH = SAVE_PATH
with open(SAVE_PATH, 'rb') as f:
    model = torch.load(f)

In [35]:
# Run on test data.
# update_count = 0
test_loss, n1, n100, r20, r50 = evaluate(test_data_tr, test_data_te)
print('=' * 89)
print('| End of training | test loss {:4.2f} | n1 {:4.3f} | n100 {:4.3f} | r20 {:4.3f} | '
        'r50 {:4.3f}'.format(test_loss, np.mean(n1), np.mean(n100), np.mean(r20), np.mean(r50)))
print('=' * 89)

| End of training | test loss 366.42 | n1 0.369 | n100 0.428 | r20 0.400 | r50 0.537


In [ ]:
# index items by using weights in the encoding of VAE-model

In [16]:
stdict = model.state_dict()
print(stdict.keys())

odict_keys(['q_layers.0.weight', 'q_layers.0.bias', 'q_layers.1.weight', 'q_layers.1.bias', 'p_layers.0.weight', 'p_layers.0.bias', 'p_layers.1.weight', 'p_layers.1.bias'])


In [17]:
# B(Az+b)+b' = BAz + Bb + b' = np.column_stack((BA,Bb+b'))*np.append(z,1)

In [18]:
P0 = stdict['p_layers.0.weight']
p0_bias = stdict['p_layers.0.bias']
P1 = stdict['p_layers.1.weight']
p1_bias = stdict['p_layers.1.bias']
print(P0.shape)
print(p0_bias.shape)
print(P1.shape)
print(p1_bias.shape)

torch.Size([600, 200])
torch.Size([600])
torch.Size([20101, 600])
torch.Size([20101])


In [19]:
# np.matmul(P1,np.tanh((np.matmul(P0,z)+P0_bias)))+P1_bias

In [20]:
# B(tanh(Az+b))+b' = Bz'+b' = ([B,b'](z',1))
P1_dash = torch.column_stack((P1,p1_bias))
print(P1_dash.shape)

torch.Size([20101, 601])


In [21]:
# print(P0.shape)
# print(P0_bias.shape)
# print(torch.matmul(z_gpu,P0.T).to(device)[5])
# print(torch.matmul(z_gpu,P0.T).to(device).shape)
# print(P0_bias.to(device)[5])
# print((torch.matmul(P0,z_gpu)+P0_bias).to(device)[5])
# print(torch.add(torch.matmul(P0,z_gpu),P0_bias).to(device)[5])

In [22]:
import faiss
# use a single GPU
res = faiss.StandardGpuResources()
# # use a multi GPUs
# ngpus = faiss.get_num_gpus()

# build a flat (CPU) index
d = P1_dash.shape[1]
index_flat = faiss.IndexFlatIP(d)
# gpu_index_flat = faiss.GpuIndexFlatIP(res,d)

# make it into a gpu index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
# # make it into a gpu index(multi GPUs)
# gpu_index = faiss.index_cpu_to_all_gpus(  # build the index
#     cpu_index
# )

# indexing
gpu_index_flat.add(P1_dash.cpu())
# gpu_index_flat.add(P1_dash) 
print(gpu_index_flat.ntotal)

20101


In [23]:
nlist = 100
quantizer = faiss.IndexFlatIP(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)
gpu_index_ivf = faiss.index_cpu_to_gpu(res, 0, index_ivf)
gpu_index_ivf.train(P1_dash.cpu())
gpu_index_ivf.add(P1_dash.cpu())

gpu_index_ivf.nprobe = 3

In [24]:
# P0 = torch.FloatTensor(P0).to(device)
# P0_bias = torch.FloatTensor(P0_bias).to(device)

In [25]:
# update_count = 

# tauをアイテム数で決める(gumbel-sharpをアイテム数で決めてみる)

In [84]:
# beta = np.log(1/n_items)
beta = 1
# tau = 0.1
tau = 1/np.log(np.log(n_items))
print(tau)
print(1/tau)

0.43603469522020893
2.293395481969557


In [85]:
def sampling_ranking(scores, heldout_data, seed, n1_list_per_sampling, n100_list_per_sampling, r20_list_per_sampling, r50_list_per_sampling):
    # Add Gumbel samples
    np.random.seed(seed=seed)
    gumbel_sampled_scores = scores + np.vectorize(gumbel_inverse)(np.random.uniform(size=scores.shape))
    # Exclude examples from training set
    # gumbel_sampled_scores[data.nonzero()] = -np.inf

    n1_list_per_sampling.append(metric.NDCG_binary_at_k_batch(gumbel_sampled_scores, heldout_data, 1))
    n100_list_per_sampling.append(metric.NDCG_binary_at_k_batch(gumbel_sampled_scores, heldout_data, 100))
    r20_list_per_sampling.append(metric.Recall_at_k_batch(gumbel_sampled_scores, heldout_data, 20))
    r50_list_per_sampling.append(metric.Recall_at_k_batch(gumbel_sampled_scores, heldout_data, 50))

In [87]:
def evaluate_expectation2(data_tr, data_te, n_sampling=1):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    n1_list = []
    n100_list = []
    r20_list = []
    r50_list = []
    n1_list_per_sampling = []
    n100_list_per_sampling = []
    r20_list_per_sampling = []
    r50_list_per_sampling = []
    
    manager = Manager()
    dummy = manager.dict()
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]

                data_tensor = naive_sparse2tensor(data).to(device)

                if TOTAL_ANNEAL_STEPS > 0:
                    anneal = min(ANNEAL_CAP, 
                                   1. * update_count / TOTAL_ANNEAL_STEPS)
                else:
                    anneal = ANNEAL_CAP

                recon_batch, mu, logvar = model(data_tensor)

                loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
                total_loss += loss.item()
                # pbar.set_description(OrderedDict(total_loss=total_loss))

                # print(torch.mean(recon_batch,1))
                # print(torch.transpose(recon_batch,0,1).size())
                # print(torch.mean(torch.transpose(torch.div(torch.transpose(recon_batch,0,1), torch.mean(recon_batch,1)),0,1),1))
                # print(torch.div(torch.transpose(torch.div(torch.transpose(recon_batch,0,1), torch.mean(recon_batch,1)),0,1),tau).size())
                # print(torch.mean(torch.div(torch.transpose(torch.div(torch.transpose(recon_batch,0,1), torch.mean(recon_batch,1)),0,1),tau),1))

                recon_batch = F.log_softmax(torch.div(recon_batch,tau), 1)
                # recon_batch = F.log_softmax(torch.div(torch.transpose(torch.div(torch.transpose(recon_batch,0,1), torch.mean(recon_batch,1)),0,1),tau), 1)
                recon_batch = recon_batch.cpu().numpy()
                # recon_batch[data.nonzero()] = -np.inf
                
                with Manager() as manager:
                    # d = manager.dict()
                    # l = manager.list()
                    n1_list_per_sampling = manager.list()
                    n100_list_per_sampling = manager.list()
                    r20_list_per_sampling = manager.list()
                    r50_list_per_sampling = manager.list()
                    p_list = []
                    for l in range(n_sampling):
                        p = Process(target=sampling_ranking, args=(recon_batch,heldout_data,l,n1_list_per_sampling,n100_list_per_sampling,r20_list_per_sampling,r50_list_per_sampling))
                        p.start()
                        p_list.append(p)
                        if len(p_list) % 4 == 0:
                            for p in p_list:
                                p.join()
                            p_list = []
                    
                    for p in p_list:
                        p.join()
                    
                    # print(n1_list_per_sampling)
                    # print(n100_list_per_sampling)

                    n1_list.append(np.concatenate(n1_list_per_sampling))
                    n100_list.append(np.concatenate(n100_list_per_sampling))
                    r20_list.append(np.concatenate(r20_list_per_sampling))
                    r50_list.append(np.concatenate(r50_list_per_sampling))
    
    total_loss /= len(range(0, e_N, BATCH_SIZE))
    n1_list = np.concatenate(n1_list)
    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)

    return total_loss, n1_list, n100_list, r20_list, r50_list

In [88]:
# Run on test data.
test_loss2, n1_list2, n100_list2, r20_list2, r50_list2 = evaluate_expectation2(test_data_tr, test_data_te, n_sampling=20)
print('=' * 89)
print('| End of training | test loss {:4.2f} | n1 {:4.3f}({:4.3f}) | n100 {:4.3f}({:4.3f}) | r20 {:4.3f}({:4.3f}) | '
        'r50 {:4.3f}({:4.3f})'.format(test_loss2, np.mean(n1_list2), np.std(n1_list2)/np.sqrt(len(n1_list2)), np.mean(n100_list2), np.std(n100_list2)/np.sqrt(len(n100_list2)), np.mean(r20_list2), np.std(r20_list2)/np.sqrt(len(r20_list2)), np.mean(r50_list2), np.std(r50_list2)/np.sqrt(len(r50_list2))))
print('=' * 89)

[test]: 100%|██████████| 20/20 [2:08:43<00:00, 386.16s/it]t]

| End of training | test loss 366.42 | n1 0.068(0.001) | n100 0.247(0.000) | r20 0.201(0.001) | r50 0.386(0.001)


# stochastic VAE

In [28]:
def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

def evaluate_stochasticVAE(data_tr, data_te, n_sampling=1):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    n1_list = []
    n100_list = []
    r20_list = []
    r50_list = []
    n1_list_per_sampling = []
    n100_list_per_sampling = []
    r20_list_per_sampling = []
    r50_list_per_sampling = []
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]

                data_tensor = naive_sparse2tensor(data).to(device)

                if TOTAL_ANNEAL_STEPS > 0:
                    anneal = min(ANNEAL_CAP, 
                                   1. * update_count / TOTAL_ANNEAL_STEPS)
                else:
                    anneal = ANNEAL_CAP

                # recon_batch, mu, logvar = model(data_tensor)
                mu, logvar = model.encode(data_tensor)

                # loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
                # total_loss += loss.item()
                # recon_batch = recon_batch.cpu().numpy()
                # recon_batch[data.nonzero()] = -np.inf

                for l in range(n_sampling):
                    z = reparameterize(mu, logvar)
                    # print(z)
                    recon_batch = model.decode(z)
                    recon_batch = recon_batch.cpu().numpy()

                    n1_list_per_sampling.append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 1))
                    n100_list_per_sampling.append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100))
                    r20_list_per_sampling.append(metric.Recall_at_k_batch(recon_batch, heldout_data, 20))
                    r50_list_per_sampling.append(metric.Recall_at_k_batch(recon_batch, heldout_data, 50))

                n1_list.append(np.concatenate(n1_list_per_sampling))
                n100_list.append(np.concatenate(n100_list_per_sampling))
                r20_list.append(np.concatenate(r20_list_per_sampling))
                r50_list.append(np.concatenate(r50_list_per_sampling))
    
    total_loss /= len(range(0, e_N, BATCH_SIZE))
    n1_list = np.concatenate(n1_list)
    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)

    return total_loss, n1_list, n100_list, r20_list, r50_list

In [25]:
test_loss, n1_list, n100_list, r20_list, r50_list = evaluate_stochasticVAE(test_data_tr, test_data_te, n_sampling=20)
print('=' * 89)
print('| End of training | test loss {:4.2f} | n1 {:4.3f}({:4.3f}) | n100 {:4.3f}({:4.3f}) | r20 {:4.3f}({:4.3f}) | '
        'r50 {:4.3f}({:4.3f})'.format(test_loss, np.mean(n1_list), np.std(n1_list)/np.sqrt(len(n1_list)), np.mean(n100_list), np.std(n100_list)/np.sqrt(len(n100_list)), np.mean(r20_list), np.std(r20_list)/np.sqrt(len(r20_list)), np.mean(r50_list), np.std(r50_list)/np.sqrt(len(r50_list))))
print('=' * 89)

[test]: 100%|██████████| 20/20 [02:07<00:00,  6.37s/it]

| End of training | test loss 0.00 | n1 0.059(0.000) | n100 0.254(0.000) | r20 0.217(0.000) | r50 0.395(0.000)


# evaluate multi-VAE, Gumbel-VAE, Stochastic-VAE

In [21]:
beta = 0.2
# beta = 1
# beta_dash = 0.2

def gumbel_inverse(x):
    return -beta*np.log(-np.log(x))

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

def evaluate_stochastic(data_tr, data_te, n_sampling=1):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    
    metrics = {"ndcg@20":[[] for _ in range(n_sampling)],
               # "ndcg@100":[[] for _ in range(n_sampling)],
               "recall@20":[[] for _ in range(n_sampling)],
               # "recall@50":[[] for _ in range(n_sampling)],
               "precision@20":[[] for _ in range(n_sampling)],
               # "precision@50":[[] for _ in range(n_sampling)],
               "hit_rate@20" : [[] for _ in range(n_sampling)],
               # "hit_rate@100" : [[] for _ in range(n_sampling)],
               "prediction_time": [[] for _ in range(n_sampling)],
              }
    
    metrics_dic = {
        "multi-VAE":copy.deepcopy(metrics),
        "multi-VAE-Faiss":copy.deepcopy(metrics),
        "multi-VAE-Faiss-IVF":copy.deepcopy(metrics),
        "multi-VAE-Gumbel":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-low-beta":copy.deepcopy(metrics),
        "multi-VAE-Stochastic":copy.deepcopy(metrics),
        "multi-VAE-Stochastic-Faiss":copy.deepcopy(metrics),
        "multi-VAE-Stochastic-Faiss-IVF":copy.deepcopy(metrics),
                }
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]

                data_tensor = naive_sparse2tensor(data).to(device)
                
                n_batch_user = data.shape[0]
                non_zero_indices = data.nonzero()

                # if TOTAL_ANNEAL_STEPS > 0:
                #     anneal = min(ANNEAL_CAP, 
                #                    1. * update_count / TOTAL_ANNEAL_STEPS)
                # else:
                #     anneal = ANNEAL_CAP
                
                non_zero_indices = np.split(data.indices, data.indptr)

                # encoding
                start = time.perf_counter()
                mu, logvar = model.encode(data_tensor)
                t_encode = time.perf_counter() - start
                
                # decoding
                start = time.perf_counter()
                recon_batch = model.decode(mu)  
                t_decode = time.perf_counter() - start
                
                recon_batch_clone = recon_batch.clone()
                
                start = time.perf_counter()
                recon_batch = recon_batch.cpu()
                recon_batch = recon_batch.numpy()
                # recon_batch[data.nonzero()] = -np.inf
                t_to_cpu = time.perf_counter() - start

                # loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
                # total_loss += loss.item()
                # recon_batch = recon_batch.cpu().numpy()
                
                # https://stackoverflow.com/questions/59338537/summarize-non-zero-values-in-a-scipy-matrix-by-axis
                n_already_intaract_item = data.indptr[1:] - data.indptr[:-1]
                max_n_already_intaract_item = int(np.max(n_already_intaract_item))

                for l in range(n_sampling):
                    torch.manual_seed(l)
                    
                    # bluring z
                    start = time.perf_counter()
                    z_blurred = reparameterize(mu, logvar)
                    t_blurred = time.perf_counter() - start
                    
                    # Stochastic multi-VAE
                    start = time.perf_counter()
                    recon_batch_blurred = model.decode(z_blurred)
                    recon_batch_blurred = recon_batch_blurred.cpu()
                    recon_batch_blurred = recon_batch_blurred.numpy()
                    # recon_batch_blurred[non_zero_indices] = -np.inf
                    t_decode_blurred = time.perf_counter() - start
                    
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch_blurred, max_n_already_intaract_item+20)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    metrics_dic["multi-VAE-Stochastic"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)
                    
                    
                    # Stochastic multi-VAE with Faiss NNS
                    start = time.perf_counter()
                    z_dash_blurred = torch.tanh(torch.add(torch.matmul(z_blurred,P0.T),p0_bias))
                    z_dash_blurred_wi_constant = torch.column_stack((z_dash_blurred,torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_flat.search(z_dash_blurred_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch_by_topk_indexes(topk_indexes, heldout_data, n_batch_user, 20))
                    metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss"]["prediction_time"][l].append((t_encode + t_blurred + t_k_nns)/n_batch_user)
                    
                    
                    # Stochastic multi-VAE with Faiss NNS(IVF)
                    start = time.perf_counter()
                    z_dash_blurred = torch.tanh(torch.add(torch.matmul(z_blurred,P0.T),p0_bias))
                    z_dash_blurred_wi_constant = torch.column_stack((z_dash_blurred,torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_ivf.search(z_dash_blurred_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch_by_topk_indexes(topk_indexes, heldout_data, n_batch_user, 20))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["prediction_time"][l].append((t_encode + t_blurred + t_k_nns)/n_batch_user)
                    
                    # multi-VAE + Gumbel Max Sampling
                    start = time.perf_counter() 
                    # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
                    # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
                    # recon_batch_gumbel_sampled = recon_batch - torch.empty_like(recon_batch, memory_format=torch.legacy_contiguous_format).exponential_().log()
                    recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                    # recon_batch_gumbel_sampled[non_zero_indices] = -np.inf
                    t_gumbel_sampling = time.perf_counter() - start
                    
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+20)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Gumbel"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)
                    
                    # multi-VAE + Faiss
                    start = time.perf_counter()
                    z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
                    z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_flat.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
                    # multi-VAE + Faiss(IVF)
                    start = time.perf_counter()
                    z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
                    z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_ivf.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Faiss-IVF"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss-IVF"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss-IVF"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss-IVF"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss-IVF"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
                    # multi-VAE
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch, max_n_already_intaract_item+20)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["prediction_time"][l].append((t_encode + t_decode + t_to_cpu + t_get_idx_topk)/n_batch_user)
                    
    
    # total_loss /= len(range(0, e_N, BATCH_SIZE))
    
    for method_name, metrics in metrics_dic.items():
        for metric_name, metric_list in metrics.items():
            if metric_name == "prediction_time":
                continue
            for l in range(n_sampling):
                metric_list[l] = np.concatenate(metric_list[l])

    return total_loss, metrics_dic

In [22]:
# Run on test data.
total_loss, metrics_dic = evaluate_stochastic(test_data_tr, test_data_te, n_sampling=100)
# print('=' * 89)
# print('| End of training | test loss {:4.2f} | n1 {:4.3f}({:4.3f}) | n100 {:4.3f}({:4.3f}) | r20 {:4.3f}({:4.3f}) | '
#         'r50 {:4.3f}({:4.3f})'.format(test_loss, np.mean(n1_list), np.std(n1_list)/np.sqrt(len(n1_list)), np.mean(n100_list), np.std(n100_list)/np.sqrt(len(n100_list)), np.mean(r20_list), np.std(r20_list)/np.sqrt(len(r20_list)), np.mean(r50_list), np.std(r50_list)/np.sqrt(len(r50_list))))
# print('=' * 89)

[test]: 100%|██████████| 20/20 [2:07:03<00:00, 381.15s/it]t]


In [ ]:
np.std(np.array(metrics_dic["multi-VAE-Stochastic"]["ndcg@20"])[:,0])

In [ ]:
np.std(np.array(metrics_dic["multi-VAE-Gumbel"]["ndcg@20"])[:,0])

In [138]:
np.std(np.array(metrics_dic["multi-VAE-Stochastic"]["ndcg@20"])[:,0])

0.13931001977971966

In [139]:
np.std(np.array(metrics_dic["multi-VAE-Gumbel"]["ndcg@20"])[:,0])

0.09817638518083249

In [23]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
        # # results.append(np.mean(metrics_dic[method_name][metric_name]))
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0224),0.3656(0.0277),0.1759(0.0152),0.8777(0.0328),0.000284(0.000003)
multi-VAE-Faiss,0.3390(0.0224),0.3656(0.0277),0.1759(0.0152),0.8777(0.0328),0.000037(0.000001)
multi-VAE-Faiss-IVF,0.3273(0.0226),0.3467(0.0277),0.1672(0.0150),0.8620(0.0345),0.000031(0.000001)
multi-VAE-Gumbel,0.3078(0.0210),0.3506(0.0276),0.1634(0.0138),0.8722(0.0334),0.000269(0.000003)
multi-VAE-Stochastic,0.2948(0.0211),0.3269(0.0267),0.1591(0.0143),0.8534(0.0354),0.000277(0.000003)
multi-VAE-Stochastic-Faiss,0.2948(0.0211),0.3269(0.0267),0.1591(0.0143),0.8534(0.0354),0.000038(0.000001)
multi-VAE-Stochastic-Faiss-IVF,0.2893(0.0213),0.3142(0.0266),0.1538(0.0142),0.8413(0.0365),0.000031(0.000001)


In [24]:
results_top20per = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            results.append("{:4.4f}({:4.4f})".format(np.mean(np.quantile(metric_list, 0.8, axis=0)), np.std(np.quantile(metric_list, 0.8, axis=0)/np.sqrt(len(np.quantile(metric_list, 0.8, axis=0))))))
    results_top20per.append(results)
results_top20per = pd.DataFrame(results_top20per, columns=metric_names, index=method_names)
print("Top 20%")
results_top20per

Top 20%


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000284(0.000003)
multi-VAE-Faiss,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000037(0.000001)
multi-VAE-Faiss-IVF,0.3273(0.0023),0.3467(0.0028),0.1672(0.0015),0.8620(0.0035),0.000031(0.000001)
multi-VAE-Gumbel,0.3657(0.0022),0.3927(0.0028),0.1852(0.0015),0.8999(0.0030),0.000269(0.000003)
multi-VAE-Stochastic,0.3651(0.0022),0.3956(0.0028),0.1865(0.0015),0.9097(0.0029),0.000277(0.000003)
multi-VAE-Stochastic-Faiss,0.3651(0.0022),0.3956(0.0028),0.1865(0.0015),0.9097(0.0029),0.000038(0.000001)
multi-VAE-Stochastic-Faiss-IVF,0.3551(0.0022),0.3777(0.0028),0.1786(0.0015),0.8948(0.0031),0.000031(0.000001)


In [25]:
results_bottom20per = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            results.append("{:4.4f}({:4.4f})".format(np.mean(np.quantile(metric_list, 0.2, axis=0)), np.std(np.quantile(metric_list, 0.2, axis=0)/np.sqrt(len(np.quantile(metric_list, 0.2, axis=0))))))
    results_bottom20per.append(results)
results_bottom20per = pd.DataFrame(results_bottom20per, columns=metric_names, index=method_names)
print("Bottom 20%")
results_bottom20per

Bottom 20%


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000284(0.000003)
multi-VAE-Faiss,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000037(0.000001)
multi-VAE-Faiss-IVF,0.3273(0.0023),0.3467(0.0028),0.1672(0.0015),0.8620(0.0035),0.000031(0.000001)
multi-VAE-Gumbel,0.2470(0.0018),0.3065(0.0027),0.1410(0.0012),0.8453(0.0037),0.000269(0.000003)
multi-VAE-Stochastic,0.2214(0.0017),0.2560(0.0024),0.1313(0.0013),0.8022(0.0040),0.000277(0.000003)
multi-VAE-Stochastic-Faiss,0.2214(0.0017),0.2560(0.0024),0.1313(0.0013),0.8022(0.0040),0.000038(0.000001)
multi-VAE-Stochastic-Faiss-IVF,0.2208(0.0018),0.2494(0.0024),0.1287(0.0013),0.7928(0.0041),0.000031(0.000001)


In [27]:
results_top50per = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            results.append("{:4.4f}({:4.4f})".format(np.mean(np.quantile(metric_list, 0.5, axis=0)), np.std(np.quantile(metric_list, 0.5, axis=0)/np.sqrt(len(np.quantile(metric_list, 0.5, axis=0))))))
    results_top50per.append(results)
results_top50per = pd.DataFrame(results_top50per, columns=metric_names, index=method_names)
print("Top 50%")
results_top50per

Top 50%


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000284(0.000003)
multi-VAE-Faiss,0.3390(0.0023),0.3656(0.0028),0.1759(0.0015),0.8777(0.0033),0.000037(0.000001)
multi-VAE-Faiss-IVF,0.3273(0.0023),0.3467(0.0028),0.1672(0.0015),0.8620(0.0035),0.000031(0.000001)
multi-VAE-Gumbel,0.3019(0.0020),0.3477(0.0028),0.1626(0.0014),0.8736(0.0034),0.000269(0.000003)
multi-VAE-Stochastic,0.2879(0.0020),0.3251(0.0026),0.1585(0.0014),0.8637(0.0035),0.000277(0.000003)
multi-VAE-Stochastic-Faiss,0.2879(0.0020),0.3251(0.0026),0.1585(0.0014),0.8637(0.0035),0.000038(0.000001)
multi-VAE-Stochastic-Faiss-IVF,0.2834(0.0020),0.3141(0.0026),0.1534(0.0014),0.8529(0.0036),0.000031(0.000001)


In [167]:
for method_name, metrics in metrics_dic.items():
        for metric_name, metric_list in metrics.items():
            # if metric_name in ("prediction_time"):
                # print("{}\t{}:{}".format(method_name, metric_name, np.mean(np.quantile(metrics_dic[method_name][metric_name], 0.9, axis=1))))
            # if metric_name in ("hit_rate@20","hit_rate@50"):
                # print("{}\t{}:{}".format(method_name, metric_name, np.mean(np.quantile(metrics_dic[method_name][metric_name], 0.5, axis=1))))
            print("{}\t{}:{}".format(method_name, metric_name, np.mean(np.quantile(metrics_dic[method_name][metric_name], 0.99, axis=0))))

multi-VAE	ndcg@20:0.3390019526235259
multi-VAE	ndcg@100:0.427921597610972
multi-VAE	recall@20:0.36564431951977444
multi-VAE	recall@50:0.5309715538503059
multi-VAE	precision@20:0.1758791678022434
multi-VAE	precision@50:0.11315272909866547
multi-VAE	hit_rate@20:0.8777346145982416
multi-VAE	hit_rate@100:0.9597219382539358
multi-VAE	prediction_time:0.025096308455737015
multi-VAE-Gumbel	ndcg@20:0.24272735428304837
multi-VAE-Gumbel	ndcg@100:0.3353621752809509
multi-VAE-Gumbel	recall@20:0.3126150629968296
multi-VAE-Gumbel	recall@50:0.482291047211655
multi-VAE-Gumbel	precision@20:0.11779002481942416
multi-VAE-Gumbel	precision@50:0.08251114230331066
multi-VAE-Gumbel	hit_rate@20:0.9085217746882028
multi-VAE-Gumbel	hit_rate@100:0.9757963606624412
multi-VAE-Gumbel	prediction_time:0.02695225974983259
multi-VAE-Stochastic	ndcg@20:0.39673035017837077
multi-VAE-Stochastic	ndcg@100:0.4738134190134512
multi-VAE-Stochastic	recall@20:0.42287913535252136
multi-VAE-Stochastic	recall@50:0.5839383764597308
mu

In [66]:
beta = 0.2
# beta = 1
# beta_dash = 0.2

def gumbel_inverse(x):
    return -beta*np.log(-np.log(x))

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

def evaluate_stochastic(data_tr, data_te, n_sampling=1):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    
    metrics = {"ndcg@20":[[] for _ in range(n_sampling)],
               # "ndcg@100":[[] for _ in range(n_sampling)],
               "recall@20":[[] for _ in range(n_sampling)],
               # "recall@50":[[] for _ in range(n_sampling)],
               "precision@20":[[] for _ in range(n_sampling)],
               # "precision@50":[[] for _ in range(n_sampling)],
               "hit_rate@20" : [[] for _ in range(n_sampling)],
               # "hit_rate@100" : [[] for _ in range(n_sampling)],
               "prediction_time": [[] for _ in range(n_sampling)],
              }
    
    metrics_dic = {
        "multi-VAE":copy.deepcopy(metrics),
        # "multi-VAE-Faiss":copy.deepcopy(metrics),
        # "multi-VAE-Faiss-IVF":copy.deepcopy(metrics),
        "multi-VAE-Gumbel":copy.deepcopy(metrics),
        "multi-VAE-Gumbel-k":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-low-beta":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-k":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss-k":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss-IVF":copy.deepcopy(metrics),
                }
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]

                data_tensor = naive_sparse2tensor(data).to(device)
                
                n_batch_user = data.shape[0]
                non_zero_indices = data.nonzero()

                # if TOTAL_ANNEAL_STEPS > 0:
                #     anneal = min(ANNEAL_CAP, 
                #                    1. * update_count / TOTAL_ANNEAL_STEPS)
                # else:
                #     anneal = ANNEAL_CAP
                
                non_zero_indices = np.split(data.indices, data.indptr)

                # encoding
                start = time.perf_counter()
                mu, logvar = model.encode(data_tensor)
                t_encode = time.perf_counter() - start
                
                # decoding
                start = time.perf_counter()
                recon_batch = model.decode(mu)  
                t_decode = time.perf_counter() - start
                
                recon_batch_clone = recon_batch.clone()
                
                start = time.perf_counter()
                recon_batch = recon_batch.cpu()
                recon_batch = recon_batch.numpy()
                # recon_batch[data.nonzero()] = -np.inf
                t_to_cpu = time.perf_counter() - start

                # loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
                # total_loss += loss.item()
                # recon_batch = recon_batch.cpu().numpy()
                
                # https://stackoverflow.com/questions/59338537/summarize-non-zero-values-in-a-scipy-matrix-by-axis
                n_already_intaract_item = data.indptr[1:] - data.indptr[:-1]
                max_n_already_intaract_item = int(np.max(n_already_intaract_item))

                for l in range(n_sampling):
                    torch.manual_seed(l)
                    
#                     # bluring z
#                     start = time.perf_counter()
#                     z_blurred = reparameterize(mu, logvar)
#                     t_blurred = time.perf_counter() - start
                    
#                     # Stochastic multi-VAE
#                     start = time.perf_counter()
#                     recon_batch_blurred = model.decode(z_blurred)
#                     recon_batch_blurred = recon_batch_blurred.cpu()
#                     recon_batch_blurred = recon_batch_blurred.numpy()
#                     # recon_batch_blurred[non_zero_indices] = -np.inf
#                     t_decode_blurred = time.perf_counter() - start
                    
#                     start = time.perf_counter()
#                     topk_indexes = metric.get_idx_topk(recon_batch_blurred, max_n_already_intaract_item+20)
#                     t_get_idx_topk = time.perf_counter() - start
                    
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf

#                     metrics_dic["multi-VAE-Stochastic"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)
                    
#                     # Stochastic multi-VAE (add noise for each position in a ranking)
#                     start = time.perf_counter()
#                     topk_indexes = np.empty((n_batch_user,0),dtype=int)
#                     data_tmp = data.copy()
#                     for i in range(20):
#                         z_blurred = reparameterize(mu, logvar)
#                         recon_batch_blurred = model.decode(z_blurred)
#                         recon_batch_blurred = recon_batch_blurred.cpu()
#                         recon_batch_blurred = recon_batch_blurred.numpy()
#                         recon_batch_blurred[data_tmp.nonzero()] = -np.inf
#                         topk_indexes_tmp = metric.get_idx_topk(recon_batch_blurred, 1)
#                         # print(topk_indexes_tmp[0])
#                         topk_indexes = np.concatenate((topk_indexes, topk_indexes_tmp), axis=1)
#                         data_tmp += coo_matrix((np.ones(n_batch_user, dtype=np.int8), (np.arange(n_batch_user), topk_indexes_tmp.flatten())), shape=data_tmp.shape)
#                         # topk_indexes = np.array([list(OrderedDict.fromkeys(row))[:i+1] for row in topk_indexes])
#                         # print(topk_indexes[0])
#                         # print("---")
#                     t_get_idx_topk = time.perf_counter() - start
                    
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,20+1), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     # recon_batch_dummy[data.nonzero()] = -np.inf

#                     metrics_dic["multi-VAE-Stochastic-k"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-k"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-k"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-k"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-k"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)
                    
                    # multi-VAE + Gumbel Max Sampling
                    start = time.perf_counter() 
                    # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
                    # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
                    # recon_batch_gumbel_sampled = recon_batch - torch.empty_like(recon_batch, memory_format=torch.legacy_contiguous_format).exponential_().log()
                    beta = 0.2
                    recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                    # recon_batch_gumbel_sampled[non_zero_indices] = -np.inf
                    t_gumbel_sampling = time.perf_counter() - start
                    
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+20)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Gumbel"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)
                    
                    # multi-VAE + Gumbel Max Sampling (add noise for each position in a ranking)
                    start = time.perf_counter() 
                    # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
                    # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
                    topk_indexes = np.empty((n_batch_user,0),dtype=int)
                    data_tmp = data.copy()
                    for i in range(20):
                        beta = i * 0.02
                        recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
                        recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
                        recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                        recon_batch_gumbel_sampled[data_tmp.nonzero()] = -np.inf
                        # topk_indexes_tmp = metric.get_idx_topk(recon_batch_gumbel_sampled, i+1)
                        topk_indexes_tmp = metric.get_idx_topk(recon_batch_gumbel_sampled, 1)
                        topk_indexes = np.concatenate((topk_indexes, topk_indexes_tmp), axis=1) 
                        data_tmp += coo_matrix((np.ones(n_batch_user, dtype=np.int8), (np.arange(n_batch_user), topk_indexes_tmp.flatten())), shape=data_tmp.shape)
                        # topk_indexes = np.array([list(OrderedDict.fromkeys(row))[:i+1] for row in topk_indexes])
                        # print(topk_indexes[0])
                        
                    # recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
                    # recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                    
                    # t_gumbel_sampling = time.perf_counter() - start
                    
                    # start = time.perf_counter()
                    # topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+20)
                        
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,21), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Gumbel-k"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel-k"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel-k"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel-k"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel-k"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)
                    
#                     # multi-VAE + Faiss
#                     start = time.perf_counter()
#                     z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
#                     z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
#                     topk_scores, topk_indexes = gpu_index_flat.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
#                     # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
#                     t_k_nns = time.perf_counter() - start
                    
#                     # create dummy recon_batch which include the corresponded score to top-k items
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf
                    
#                     metrics_dic["multi-VAE-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
#                     # multi-VAE + Faiss(IVF)
#                     start = time.perf_counter()
#                     z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
#                     z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
#                     topk_scores, topk_indexes = gpu_index_ivf.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
#                     # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
#                     t_k_nns = time.perf_counter() - start
                    
#                     # create dummy recon_batch which include the corresponded score to top-k items
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf
                    
#                     metrics_dic["multi-VAE-Faiss-IVF"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss-IVF"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss-IVF"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss-IVF"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss-IVF"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
                    # multi-VAE
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch, max_n_already_intaract_item+20)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
                    # metrics_dic["multi-VAE"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["prediction_time"][l].append((t_encode + t_decode + t_to_cpu + t_get_idx_topk)/n_batch_user)
                    
    
    # total_loss /= len(range(0, e_N, BATCH_SIZE))
    
    for method_name, metrics in metrics_dic.items():
        for metric_name, metric_list in metrics.items():
            if metric_name == "prediction_time":
                continue
            for l in range(n_sampling):
                metric_list[l] = np.concatenate(metric_list[l])

    return total_loss, metrics_dic

In [67]:
total_loss, metrics_dic = evaluate_stochastic(test_data_tr, test_data_te, n_sampling=5)

[test]: 100%|██████████| 20/20 [05:48<00:00, 17.43s/it]


In [121]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metrics_dic[method_name][metric_name]), np.std(metrics_dic[method_name][metric_name])))
        else:
        # # results.append(np.mean(metrics_dic[method_name][metric_name]))
            results.append("{:4.4f}({:4.4f})".format(np.mean(metrics_dic[method_name][metric_name]), np.std(metrics_dic[method_name][metric_name])))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.2239),0.3656(0.2772),0.1759(0.1524),0.8777(0.3276),0.000281(0.000030)
multi-VAE-Faiss,0.3390(0.2239),0.3656(0.2772),0.1759(0.1524),0.8777(0.3276),0.000034(0.000007)
multi-VAE-Faiss-IVF,0.3273(0.2258),0.3467(0.2772),0.1672(0.1499),0.8620(0.3449),0.000028(0.000004)
multi-VAE-Gumbel,0.3078(0.2102),0.3508(0.2763),0.1635(0.1382),0.8720(0.3340),0.000266(0.000030)
multi-VAE-Gumbel-k,0.3080(0.2104),0.3507(0.2762),0.1636(0.1383),0.8718(0.3343),0.003902(0.000056)
multi-VAE-Stochastic,0.2946(0.2107),0.3267(0.2664),0.1591(0.1428),0.8537(0.3534),0.000274(0.000030)
multi-VAE-Stochastic-k,0.3001(0.2085),0.3408(0.2731),0.1598(0.1368),0.8655(0.3412),0.004041(0.000058)


In [65]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metrics_dic[method_name][metric_name]), np.std(metrics_dic[method_name][metric_name])))
        else:
        # # results.append(np.mean(metrics_dic[method_name][metric_name]))
            results.append("{:4.4f}({:4.4f})".format(np.mean(metrics_dic[method_name][metric_name]), np.std(metrics_dic[method_name][metric_name])))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.2239),0.3656(0.2772),0.1759(0.1524),0.8777(0.3276),0.000276(0.000030)
multi-VAE-Gumbel,0.3084(0.2113),0.3509(0.2772),0.1632(0.1379),0.8723(0.3338),0.000273(0.000116)
multi-VAE-Gumbel-k,0.1885(0.1911),0.1579(0.2049),0.0635(0.0609),0.6830(0.4653),0.004033(0.000133)


In [122]:
results_top20per = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)))
        else:
            results.append("{:4.4f}({:4.4f})".format(np.mean(np.quantile(metric_list, 0.8, axis=0)), np.std(np.quantile(metric_list, 0.8, axis=0))))
            # results.append(np.mean(np.quantile(metrics_dic[method_name][metric_name], 0.8, axis=0)))
    results_top20per.append(results)
results_top20per = pd.DataFrame(results_top20per, columns=metric_names, index=method_names)
print("Top 20%")
results_top20per

Top 20%


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.2239),0.3656(0.2772),0.1759(0.1524),0.8777(0.3276),0.000281(0.000030)
multi-VAE-Faiss,0.3390(0.2239),0.3656(0.2772),0.1759(0.1524),0.8777(0.3276),0.000034(0.000007)
multi-VAE-Faiss-IVF,0.3273(0.2258),0.3467(0.2772),0.1672(0.1499),0.8620(0.3449),0.000028(0.000004)
multi-VAE-Gumbel,0.3647(0.2189),0.3919(0.2782),0.1849(0.1485),0.8990(0.3008),0.000266(0.000030)
multi-VAE-Gumbel-k,0.3655(0.2193),0.3924(0.2780),0.1851(0.1485),0.8999(0.2994),0.003902(0.000056)
multi-VAE-Stochastic,0.3638(0.2132),0.3945(0.2728),0.1861(0.1505),0.9100(0.2851),0.000274(0.000030)
multi-VAE-Stochastic-k,0.3597(0.2143),0.3932(0.2785),0.1814(0.1439),0.9059(0.2913),0.004041(0.000058)


In [84]:
beta = 0.2
# beta = 1
# beta_dash = 0.2

def gumbel_inverse(x):
    return -beta*np.log(-np.log(x))

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

def evaluate_stochastic(data_tr, data_te, n_sampling=1, k=20):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    
    metrics = {"ndcg@{}".format(k):[[] for _ in range(n_sampling)],
               # "ndcg@100":[[] for _ in range(n_sampling)],
               "recall@{}".format(k):[[] for _ in range(n_sampling)],
               # "recall@50":[[] for _ in range(n_sampling)],
               "precision@{}".format(k):[[] for _ in range(n_sampling)],
               # "precision@50":[[] for _ in range(n_sampling)],
               "hit_rate@{}".format(k) : [[] for _ in range(n_sampling)],
               # "hit_rate@100" : [[] for _ in range(n_sampling)],
               "prediction_time": [[] for _ in range(n_sampling)],
              }
    
    metrics_dic = {
        "multi-VAE":copy.deepcopy(metrics),
        # "multi-VAE-Faiss":copy.deepcopy(metrics),
        "multi-VAE-Faiss-IVF":copy.deepcopy(metrics),
        "multi-VAE-Gumbel":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-k":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-low-beta":copy.deepcopy(metrics),
        "multi-VAE-Stochastic":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-k":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss-k":copy.deepcopy(metrics),
        "multi-VAE-Stochastic-Faiss-IVF":copy.deepcopy(metrics),
                }
    n_intaracts = []
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]

                data_tensor = naive_sparse2tensor(data).to(device)
                
                n_batch_user = data.shape[0]
                non_zero_indices = data.nonzero()

                # if TOTAL_ANNEAL_STEPS > 0:
                #     anneal = min(ANNEAL_CAP, 
                #                    1. * update_count / TOTAL_ANNEAL_STEPS)
                # else:
                #     anneal = ANNEAL_CAP
                
                non_zero_indices = np.split(data.indices, data.indptr)

                # encoding
                start = time.perf_counter()
                mu, logvar = model.encode(data_tensor)
                t_encode = time.perf_counter() - start
                
                # decoding
                start = time.perf_counter()
                recon_batch = model.decode(mu)  
                t_decode = time.perf_counter() - start
                
                recon_batch_clone = recon_batch.clone()
                
                start = time.perf_counter()
                recon_batch = recon_batch.cpu()
                recon_batch = recon_batch.numpy()
                # recon_batch[data.nonzero()] = -np.inf
                t_to_cpu = time.perf_counter() - start

                # loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)
                # total_loss += loss.item()
                # recon_batch = recon_batch.cpu().numpy()
                
                # https://stackoverflow.com/questions/59338537/summarize-non-zero-values-in-a-scipy-matrix-by-axis
                n_already_intaract_item = data.indptr[1:] - data.indptr[:-1]
                max_n_already_intaract_item = int(np.max(n_already_intaract_item))
                
                # n_intaracts.extend(n_already_intaract_item)
                n_intaracts.extend(logvar.cpu())

                for l in range(n_sampling):
                    torch.manual_seed(l)
                    
                    # bluring z
                    start = time.perf_counter()
                    z_blurred = reparameterize(mu, logvar)
                    t_blurred = time.perf_counter() - start
                    
                    # Stochastic multi-VAE
                    start = time.perf_counter()
                    recon_batch_blurred = model.decode(z_blurred)
                    recon_batch_blurred = recon_batch_blurred.cpu()
                    recon_batch_blurred = recon_batch_blurred.numpy()
                    # recon_batch_blurred[non_zero_indices] = -np.inf
                    t_decode_blurred = time.perf_counter() - start
                    
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch_blurred, max_n_already_intaract_item+k)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+k+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    metrics_dic["multi-VAE-Stochastic"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)
                    
#                     # Stochastic multi-VAE (add noise for each position in a ranking)
#                     start = time.perf_counter()
#                     topk_indexes = np.empty((n_batch_user,0),dtype=int)
#                     for i in range(20):
#                         z_blurred = reparameterize(mu, logvar)
#                         recon_batch_blurred = model.decode(z_blurred)
#                         recon_batch_blurred = recon_batch_blurred.cpu()
#                         recon_batch_blurred = recon_batch_blurred.numpy()
#                         recon_batch_blurred[data.nonzero()] = -np.inf
#                         topk_indexes_tmp = metric.get_idx_topk(recon_batch_blurred, i+1)
#                         # print(topk_indexes_tmp[0])
#                         topk_indexes = np.concatenate((topk_indexes, topk_indexes_tmp), axis=1)
#                         topk_indexes = np.array([list(OrderedDict.fromkeys(row))[:i+1] for row in topk_indexes])
#                         # print(topk_indexes[0])
#                         # print("---")
#                     t_get_idx_topk = time.perf_counter() - start
                    
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,20+1), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     # recon_batch_dummy[data.nonzero()] = -np.inf

#                     metrics_dic["multi-VAE-Stochastic-k"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-k"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-k"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-k"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-k"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)
                    
                    
#                     # Stochastic multi-VAE with Faiss NNS
#                     start = time.perf_counter()
#                     z_dash_blurred = torch.tanh(torch.add(torch.matmul(z_blurred,P0.T),p0_bias))
#                     z_dash_blurred_wi_constant = torch.column_stack((z_dash_blurred,torch.ones(n_batch_user, device=device)))
#                     topk_scores, topk_indexes = gpu_index_flat.search(z_dash_blurred_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
#                     # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
#                     t_k_nns = time.perf_counter() - start
                    
#                     # create dummy recon_batch which include the corresponded score to top-k items
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf

#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch_by_topk_indexes(topk_indexes, heldout_data, n_batch_user, 20))
#                     metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-Faiss"]["prediction_time"][l].append((t_encode + t_blurred + t_k_nns)/n_batch_user)
                    
#                     # Stochastic multi-VAE with Faiss NNS (add noise for each position in a ranking)
#                     start = time.perf_counter()
#                     topk_indexes = [[] for _ in range(n_batch_user)] 
#                     for i in range(20):
#                         z_dash_blurred = torch.tanh(torch.add(torch.matmul(z_blurred,P0.T),p0_bias))
#                         z_dash_blurred_wi_constant = torch.column_stack((z_dash_blurred,torch.ones(n_batch_user, device=device)))
#                         rank_i_scores, topk_indexes_tmp = gpu_index_flat.search(z_dash_blurred_wi_constant.cpu(), max_n_already_intaract_item+i+1) # perform searching on GPU
#                         topk_indexes_tmp = [[elem for elem in a if elem not in b] for a, b in zip(topk_indexes_tmp, non_zero_indices)]
#                         # if i == 0:
#                         #     topk_indexes = topk_indexes_tmp
#                         # else:
#                         topk_indexes = [a + b for a, b in zip(topk_indexes, topk_indexes_tmp)]
#                         topk_indexes = [list(OrderedDict.fromkeys(row))[:i+1] for row in topk_indexes]
                        
#                     topk_indexes = np.array(topk_indexes)
#                     t_k_nns = time.perf_counter() - start
                    
#                     # create dummy recon_batch which include the corresponded score to top-k items
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,21), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf

#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch_by_topk_indexes(topk_indexes, heldout_data, n_batch_user, 20))
#                     metrics_dic["multi-VAE-Stochastic-Faiss-k"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-Faiss-k"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-Faiss-k"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 50))
#                     metrics_dic["multi-VAE-Stochastic-Faiss-k"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 100))
#                     metrics_dic["multi-VAE-Stochastic-Faiss-k"]["prediction_time"][l].append((t_encode + t_blurred + t_k_nns)/n_batch_user)
                    
                    # Stochastic multi-VAE with Faiss NNS(IVF)
                    start = time.perf_counter()
                    z_dash_blurred = torch.tanh(torch.add(torch.matmul(z_blurred,P0.T),p0_bias))
                    z_dash_blurred_wi_constant = torch.column_stack((z_dash_blurred,torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_ivf.search(z_dash_blurred_wi_constant.cpu(), max_n_already_intaract_item+k) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch_by_topk_indexes(topk_indexes, heldout_data, n_batch_user, 20))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Stochastic-Faiss"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic-Faiss-IVF"]["prediction_time"][l].append((t_encode + t_blurred + t_k_nns)/n_batch_user)
                    
                    # multi-VAE + Gumbel Max Sampling
                    start = time.perf_counter() 
                    # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
                    # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
                    # recon_batch_gumbel_sampled = recon_batch - torch.empty_like(recon_batch, memory_format=torch.legacy_contiguous_format).exponential_().log()
                    recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
                    recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                    # recon_batch_gumbel_sampled[non_zero_indices] = -np.inf
                    t_gumbel_sampling = time.perf_counter() - start
                    
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+k)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+k+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Gumbel"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Gumbel"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Gumbel"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)
                    
#                     # multi-VAE + Gumbel Max Sampling (add noise for each position in a ranking)
#                     start = time.perf_counter() 
#                     # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
#                     # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
#                     topk_indexes = np.empty((n_batch_user,0),dtype=int)
#                     for i in range(20):
#                         recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
#                         recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
#                         recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
#                         recon_batch_gumbel_sampled[data.nonzero()] = -np.inf
#                         topk_indexes_tmp = metric.get_idx_topk(recon_batch_gumbel_sampled, 1)
#                         topk_indexes = np.concatenate((topk_indexes, topk_indexes_tmp), axis=1)
#                         # topk_indexes = np.array([list(OrderedDict.fromkeys(row))[:i+1] for row in topk_indexes])
#                         # topk_indexes = np.array([np.unique(row)[:i+1] for row in topk_indexes])
#                         # print(topk_indexes[0])
                        
#                     # todo 
#                     # 各行にどうやってappendしていくか
#                     # 前に選んだitem indexをどうやってtopkから除くか
                        
#                     # recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
#                     # recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
                    
#                     # t_gumbel_sampling = time.perf_counter() - start
                    
#                     # start = time.perf_counter()
#                     # topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+20)
                        
#                     t_get_idx_topk = time.perf_counter() - start
                    
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+20+1), dtype=float)), axis=1)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,21), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf
                    
#                     metrics_dic["multi-VAE-Gumbel-k"]["ndcg@20"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Gumbel-k"]["recall@20"][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Gumbel-k"]["precision@20"][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Gumbel-k"]["hit_rate@20"][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, 20))
#                     # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Gumbel-k"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)
                    
#                     # multi-VAE + Faiss
#                     start = time.perf_counter()
#                     z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
#                     z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
#                     topk_scores, topk_indexes = gpu_index_flat.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+k) # perform searching on GPU
#                     # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
#                     t_k_nns = time.perf_counter() - start
                    
#                     # create dummy recon_batch which include the corresponded score to top-k items
#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf
                    
#                     metrics_dic["multi-VAE-Faiss"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
#                     # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
#                     # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
#                     # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Faiss"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
#                     # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Faiss"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
                    # multi-VAE + Faiss(IVF)
                    start = time.perf_counter()
                    z_dash = torch.tanh(torch.add(torch.matmul(mu,P0.T),p0_bias))
                    z_dash_wi_constant = torch.column_stack((z_dash, torch.ones(n_batch_user, device=device)))
                    topk_scores, topk_indexes = gpu_index_ivf.search(z_dash_wi_constant.cpu(), max_n_already_intaract_item+20) # perform searching on GPU
                    # topk_indexes = list(map(lambda i: [ele for ele in topk_indexes[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]][:20], range(n_batch_user)))
                    t_k_nns = time.perf_counter() - start
                    
                    # create dummy recon_batch which include the corresponded score to top-k items
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, topk_scores, axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE-Faiss-IVF"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss-IVF"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss-IVF"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
                    metrics_dic["multi-VAE-Faiss-IVF"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
                    metrics_dic["multi-VAE-Faiss-IVF"]["prediction_time"][l].append((t_encode + t_k_nns)/n_batch_user)
                    
                    # multi-VAE
                    start = time.perf_counter()
                    topk_indexes = metric.get_idx_topk(recon_batch, max_n_already_intaract_item+k)
                    t_get_idx_topk = time.perf_counter() - start
                    
                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+k+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf
                    
                    metrics_dic["multi-VAE"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch, heldout_data, 50))
                    metrics_dic["multi-VAE"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, k))
                    # metrics_dic["multi-VAE"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch, heldout_data, 100))
                    metrics_dic["multi-VAE"]["prediction_time"][l].append((t_encode + t_decode + t_to_cpu + t_get_idx_topk)/n_batch_user)
                    
    
    # total_loss /= len(range(0, e_N, BATCH_SIZE))
    
    for method_name, metrics in metrics_dic.items():
        for metric_name, metric_list in metrics.items():
            if metric_name == "prediction_time":
                continue
            for l in range(n_sampling):
                metric_list[l] = np.concatenate(metric_list[l])

    return total_loss, metrics_dic, n_intaracts

In [85]:
total_loss, metrics_dic, n_intaracts = evaluate_stochastic(test_data_tr, test_data_te, n_sampling=10, k=20)

[test]: 100%|██████████| 20/20 [09:18<00:00, 27.94s/it]


In [65]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
        # # results.append(np.mean(metrics_dic[method_name][metric_name]))
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0224),0.3656(0.0277),0.1759(0.0152),0.8777(0.0328),0.000261(0.000003)
multi-VAE-Faiss-IVF,0.3273(0.0226),0.3467(0.0277),0.1672(0.0150),0.8620(0.0345),0.000028(0.000000)
multi-VAE-Gumbel,0.3078(0.0210),0.3506(0.0276),0.1634(0.0138),0.8722(0.0334),0.000256(0.000003)
multi-VAE-Stochastic,0.2948(0.0211),0.3269(0.0267),0.1591(0.0143),0.8534(0.0354),0.000264(0.000003)
multi-VAE-Stochastic-Faiss-IVF,0.2893(0.0213),0.3142(0.0266),0.1538(0.0142),0.8413(0.0365),0.000028(0.000000)


In [86]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
        # # results.append(np.mean(metrics_dic[method_name][metric_name]))
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.3390(0.0708),0.3656(0.0877),0.1759(0.0482),0.8777(0.1036),0.000278(0.000010)
multi-VAE-Faiss-IVF,0.3273(0.0714),0.3467(0.0877),0.1672(0.0474),0.8620(0.1091),0.000028(0.000001)
multi-VAE-Gumbel,0.3074(0.0664),0.3505(0.0873),0.1633(0.0436),0.8717(0.1058),0.000264(0.000009)
multi-VAE-Stochastic,0.2949(0.0667),0.3265(0.0842),0.1591(0.0452),0.8537(0.1118),0.000273(0.000009)
multi-VAE-Stochastic-Faiss-IVF,0.2894(0.0673),0.3140(0.0841),0.1538(0.0450),0.8416(0.1155),0.000035(0.000025)


In [102]:
# for i,a in enumerate(n_intaracts):
#     n_intaracts[i] = a.cpu()
# n_intaracts = np.concatenate(n_intaracts)
n_intaracts = np.exp(n_intaracts)
print(np.mean(n_intaracts))
print(np.quantile(n_intaracts, 0.1))
print(np.quantile(n_intaracts, 0.5))
print(np.quantile(n_intaracts, 0.9))

0.6365446
0.1462994322180748
0.6981931924819946
0.9902460813522339


In [67]:
len(n_intaracts)

9782

In [68]:
results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        metric_list = np.array(metric_list)
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            metric_list_low_intaracts= metric_list[:,np.array(n_intaracts)<10]
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list_low_intaracts), np.std(metric_list_low_intaracts)/np.sqrt(len(metric_list_low_intaracts))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.2323(0.0290),0.4451(0.0457),0.0308(0.0033),0.5214(0.0500),0.000261(0.000003)
multi-VAE-Faiss-IVF,0.2234(0.0292),0.4191(0.0456),0.0288(0.0032),0.4902(0.0500),0.000028(0.000000)
multi-VAE-Gumbel,0.2180(0.0278),0.4322(0.0456),0.0299(0.0032),0.5063(0.0500),0.000256(0.000003)
multi-VAE-Stochastic,0.1928(0.0271),0.3806(0.0448),0.0264(0.0032),0.4514(0.0498),0.000264(0.000003)
multi-VAE-Stochastic-Faiss-IVF,0.1877(0.0271),0.3673(0.0446),0.0254(0.0031),0.4347(0.0496),0.000028(0.000000)


In [69]:
results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        metric_list = np.array(metric_list)
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            metric_list_many_intaracts= metric_list[:,np.array(n_intaracts)>137]
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list_many_intaracts), np.std(metric_list_many_intaracts)/np.sqrt(len(metric_list_many_intaracts))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE,0.4731(0.0163),0.1677(0.0074),0.4429(0.0149),1.0000(0.0000),0.000261(0.000003)
multi-VAE-Faiss-IVF,0.4770(0.0164),0.1629(0.0071),0.4358(0.0152),0.9990(0.0032),0.000028(0.000000)
multi-VAE-Gumbel,0.4054(0.0151),0.1473(0.0069),0.3872(0.0137),0.9990(0.0032),0.000256(0.000003)
multi-VAE-Stochastic,0.4284(0.0157),0.1541(0.0069),0.4082(0.0143),0.9993(0.0027),0.000264(0.000003)
multi-VAE-Stochastic-Faiss-IVF,0.4428(0.0160),0.1527(0.0067),0.4101(0.0148),0.9988(0.0034),0.000028(0.000000)


In [156]:
from scipy.sparse import csr_matrix
import numpy as np

x = csr_matrix([[3, 0, 1], [0, 4, 0]])
array1 = np.array([[0, 1, 2], [2, 1, 0]])

# xの非ゼロ要素のインデックスを取得
non_zero_indices = x.nonzero()

# map関数とラムダ式を使用して各行で重複する要素を削除
array1_result = list(map(lambda i: [ele for ele in array1[i] if ele not in non_zero_indices[1][non_zero_indices[0] == i]], range(len(array1))))
print(array1_result)

[[1], [2, 0]]


In [170]:
a = np.array(array1_result, dtype=object)

In [175]:
a

array([list([1]), list([2, 0])], dtype=object)

In [187]:
np.reciprocal(np.array(range(1,21), dtype=float))

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
       0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
       0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667,
       0.0625    , 0.05882353, 0.05555556, 0.05263158, 0.05      ])

In [27]:
A = [[1,2],[1,3,4]]
B = [[1],[3,4]]
# AとBの各行に対して内包表記を用いて処理
A = [[elem for elem in a if elem not in b] for a, b in zip(A, B)]
print(A)
print(A[])

[[2], [1]]
<class 'list'>


In [84]:
[[] for _ in range(3)] 

[[], [], []]

In [107]:
from scipy.sparse import csr_matrix, coo_matrix

# AとBの初期化
A = csr_matrix((5, 5), dtype=np.int8)
B = np.array([[1,2],[2,3]])

# Bの各行と各要素を組み合わせてAのインデックスを作成
rows = np.repeat(np.arange(B.shape[0]), B.shape[1])
cols = B.flatten()

# 新たなスパース行列を作成
values = np.ones(rows.shape[0], dtype=np.int8)
B_sparse = coo_matrix((values, (rows, cols)), shape=A.shape)

# AにB_sparseを追加
A += B_sparse

In [108]:
rows

array([0, 0, 1, 1])

In [109]:
cols

array([1, 2, 2, 3])

In [1]:
import os
import pandas as pd
import itertools
from collections import Counter

# バイナリ行列の作成
# data = [[0, 1, 1, 0],
#         [1, 0, 1, 0],
#         [0, 1, 1, 0],
#         [1, 1, 0, 0],
#         [0, 1, 1, 1],
#         [1, 0, 1, 0]]
# df = pd.DataFrame(data)
data = train_data

# 組み合わせの大きさの範囲
r_min = 5
r_max = 10
if r_max > data.shape[1]+1:
    os.exit(0)

# 各行の1の値が含まれている列の組み合わせを求める
combinations = []
for i in range(data.shape[0]):
    row = data[i]
    ones = row.nonzero()[1]  # 非ゼロ要素の列番号を取得
    for r in range(r_min, r_max):
        combinations.extend(itertools.combinations(ones, r))
# for i in range(len(df)):
#     row = df.iloc[i]
#     ones = row[row==1].index
#     for r in range(M, len(ones)+1):
#         combinations.extend(itertools.combinations(ones, r))
        
print(combinations[:10])

# 各組み合わせが何回出現するかカウントする
counter = Counter(combinations)

# 最も頻繁に出現する組み合わせを求める
most_common_combinations = counter.most_common(1)

print(most_common_combinations)
print(counter.most_common(10))

NameError: name 'train_data' is not defined

In [54]:
from scipy.sparse import coo_matrix

beta = 0.2
# beta = 1
# beta_dash = 0.2

def gumbel_inverse(x):
    return -beta*np.log(-np.log(x))

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

def evaluate_stochastic_sampling_in_sequence(data_tr, data_te, n_sampling=1, K=20):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    
    metrics = {"ndcg@{}".format(K):[[] for _ in range(n_sampling)],
               # "ndcg@100":[[] for _ in range(n_sampling)],
               "recall@{}".format(K):[[] for _ in range(n_sampling)],
               # "recall@50":[[] for _ in range(n_sampling)],
               "precision@{}".format(K):[[] for _ in range(n_sampling)],
               # "precision@50":[[] for _ in range(n_sampling)],
               "hit_rate@{}".format(K) : [[] for _ in range(n_sampling)],
               # "hit_rate@100" : [[] for _ in range(n_sampling)],
               "prediction_time": [[] for _ in range(n_sampling)],
              }
    
    metrics_dic = {
        # "multi-VAE":copy.deepcopy(metrics),
        # "multi-VAE-Faiss":copy.deepcopy(metrics),
        # "multi-VAE-Faiss-IVF":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-k":copy.deepcopy(metrics),
        # "multi-VAE-Gumbel-low-beta":copy.deepcopy(metrics),
        "multi-VAE-Stochastic":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-k":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss-k":copy.deepcopy(metrics),
        # "multi-VAE-Stochastic-Faiss-IVF":copy.deepcopy(metrics),
                }
    n_intaracts = []
    
    with torch.no_grad():
        with tqdm(range(0, e_N, BATCH_SIZE)) as pbar:
        # for start_idx in tqdm(range(0, e_N, BATCH_SIZE)):
            for start_idx in pbar:
                pbar.set_description("[test]")
                  
                end_idx = min(start_idx + BATCH_SIZE, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                u_idxlist_wo_any_iteracts = [i for i, x in enumerate(heldout_data.toarray().sum(axis=1)) if x >0]
                data = data[u_idxlist_wo_any_iteracts]
                heldout_data = heldout_data[u_idxlist_wo_any_iteracts]
                
                n_batch_user = data.shape[0]
                # non_zero_indices = np.split(data.indices, data.indptr)
                # https://stackoverflow.com/questions/59338537/summarize-non-zero-values-in-a-scipy-matrix-by-axis
                n_already_intaract_item = data.indptr[1:] - data.indptr[:-1]
                max_n_already_intaract_item = int(np.max(n_already_intaract_item))
                n_intaracts.extend(n_already_intaract_item)
                
                for l in range(n_sampling):
                    torch.manual_seed(l)
                    topk_indexes = np.empty((n_batch_user,0),dtype=int)
                    data_tmp = data.copy()
                    
                    t_encode = 0
                    t_blurred = 0
                    t_decode_blurred = 0
                    t_get_idx_topk = 0
                    
                    for k in range(1,K+1):
                        # non_zero_indices = np.split(data_tmp.indices, data_tmp.indptr)
                        data_tensor = naive_sparse2tensor(data_tmp).to(device)

                        # encoding
                        start = time.perf_counter()
                        mu, logvar = model.encode(data_tensor)
                        t_encode += time.perf_counter() - start

#                         # decoding
#                         start = time.perf_counter()
#                         recon_batch = model.decode(mu)  
#                         t_decode = time.perf_counter() - start

#                         recon_batch_clone = recon_batch.clone()

#                         start = time.perf_counter()
#                         recon_batch = recon_batch.cpu()
#                         recon_batch = recon_batch.numpy()
#                         # recon_batch[data.nonzero()] = -np.inf
#                         t_to_cpu = time.perf_counter() - start
                    
                        # bluring z
                        start = time.perf_counter()
                        z_blurred = reparameterize(mu, logvar)
                        t_blurred += time.perf_counter() - start

                        # Stochastic multi-VAE
                        start = time.perf_counter()
                        recon_batch_blurred = model.decode(z_blurred)
                        recon_batch_blurred = recon_batch_blurred.cpu()
                        recon_batch_blurred = recon_batch_blurred.numpy()
                        recon_batch_blurred[data_tmp.nonzero()] = -np.inf
                        t_decode_blurred += time.perf_counter() - start

                        start = time.perf_counter()
                        top_indexes = metric.get_idx_topk(recon_batch_blurred, 1)
                        t_get_idx_topk += time.perf_counter() - start
                        
                        # print(top_indexes.shape)
                        # print(data_tmp.shape)
                        
                        topk_indexes = np.concatenate((topk_indexes, top_indexes), axis=1)
                        data_tmp += coo_matrix((np.ones(n_batch_user, dtype=np.int8), (np.arange(n_batch_user), top_indexes.flatten())), shape=data_tmp.shape)

                    recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
                    np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,K+1), dtype=float)), axis=1)
                    # np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(topk_dists), axis=1)
                    recon_batch_dummy[data.nonzero()] = -np.inf

                    metrics_dic["multi-VAE-Stochastic"]["ndcg@{}".format(K)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, K))
                    # metrics_dic["multi-VAE-Stochastic"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["recall@{}".format(K)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, K))
                    # metrics_dic["multi-VAE-Stochastic"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["precision@{}".format(K)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, K))
                    # metrics_dic["multi-VAE-Stochastic"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_blurred, heldout_data, 50))
                    metrics_dic["multi-VAE-Stochastic"]["hit_rate@{}".format(K)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, K))
                    # metrics_dic["multi-VAE-Stochastic"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_blurred, heldout_data, 100))
                    metrics_dic["multi-VAE-Stochastic"]["prediction_time"][l].append((t_encode + t_blurred + t_decode_blurred + t_get_idx_topk)/n_batch_user)

#                     # multi-VAE + Gumbel Max Sampling
#                     start = time.perf_counter() 
#                     # recon_batch_gumbel_sampled = recon_batch + np.vectorize(gumbel_inverse)(np.random.uniform(size=recon_batch.shape))
#                     # https://pytorch.org/docs/stable/_modules/torch/nn/functional.html#gumbel_softmax
#                     # recon_batch_gumbel_sampled = recon_batch - torch.empty_like(recon_batch, memory_format=torch.legacy_contiguous_format).exponential_().log()
#                     recon_batch_gumbel_sampled = recon_batch_clone - beta * (-torch.rand(recon_batch_clone.shape, device=device).log()).log()
#                     recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.cpu()
#                     recon_batch_gumbel_sampled = recon_batch_gumbel_sampled.numpy()
#                     # recon_batch_gumbel_sampled[non_zero_indices] = -np.inf
#                     t_gumbel_sampling = time.perf_counter() - start

#                     start = time.perf_counter()
#                     topk_indexes = metric.get_idx_topk(recon_batch_gumbel_sampled, max_n_already_intaract_item+k)
#                     t_get_idx_topk = time.perf_counter() - start

#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+k+1), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf

#                     metrics_dic["multi-VAE-Gumbel"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE-Gumbel"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Gumbel"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE-Gumbel"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Gumbel"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE-Gumbel"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 50))
#                     metrics_dic["multi-VAE-Gumbel"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE-Gumbel"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch_gumbel_sampled, heldout_data, 100))
#                     metrics_dic["multi-VAE-Gumbel"]["prediction_time"][l].append((t_encode + t_decode + t_gumbel_sampling + t_get_idx_topk)/n_batch_user)

#                     # multi-VAE
#                     start = time.perf_counter()
#                     topk_indexes = metric.get_idx_topk(recon_batch, max_n_already_intaract_item+k)
#                     t_get_idx_topk = time.perf_counter() - start

#                     recon_batch_dummy = np.ones((n_batch_user,n_items)) * (-np.inf)
#                     np.put_along_axis(recon_batch_dummy, topk_indexes, np.reciprocal(np.array(range(1,max_n_already_intaract_item+k+1), dtype=float)), axis=1)
#                     # np.put_along_axis(recon_batch_dummy, topk_indexes, np.array(range(1,21), dtype=float), axis=1)
#                     recon_batch_dummy[data.nonzero()] = -np.inf

#                     metrics_dic["multi-VAE"]["ndcg@{}".format(k)][l].append(metric.NDCG_binary_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE"]["ndcg@100"][l].append(metric.NDCG_binary_at_k_batch(recon_batch, heldout_data, 100))
#                     metrics_dic["multi-VAE"]["recall@{}".format(k)][l].append(metric.Recall_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE"]["recall@50"][l].append(metric.Recall_at_k_batch(recon_batch, heldout_data, 50))
#                     metrics_dic["multi-VAE"]["precision@{}".format(k)][l].append(metric.Precision_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE"]["precision@50"][l].append(metric.Precision_at_k_batch(recon_batch, heldout_data, 50))
#                     metrics_dic["multi-VAE"]["hit_rate@{}".format(k)][l].append(metric.HitRate_at_k_batch(recon_batch_dummy, heldout_data, K))
#                     # metrics_dic["multi-VAE"]["hit_rate@100"][l].append(metric.HitRate_at_k_batch(recon_batch, heldout_data, 100))
#                     metrics_dic["multi-VAE"]["prediction_time"][l].append((t_encode + t_decode + t_to_cpu + t_get_idx_topk)/n_batch_user)

    
    # total_loss /= len(range(0, e_N, BATCH_SIZE))
    
    for method_name, metrics in metrics_dic.items():
        for metric_name, metric_list in metrics.items():
            if metric_name == "prediction_time":
                continue
            for l in range(n_sampling):
                metric_list[l] = np.concatenate(metric_list[l])

    return total_loss, metrics_dic, n_intaracts

In [ ]:
total_loss, metrics_dic, n_intaracts = evaluate_stochastic_sampling_in_sequence(test_data_tr, test_data_te, n_sampling=5, K=20)

[test]:  90%|█████████ | 18/20 [04:45<00:31, 15.86s/it]

In [49]:
method_names = list(metrics_dic.keys())
metric_names = list(metrics_dic[method_names[0]].keys())

results_all = []
for method_name, metrics in metrics_dic.items():
    results = []
    for metric_name, metric_list in metrics.items():
        metric_list = np.array(metric_list)
        if metric_name == "prediction_time":
            results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
        else:
            metric_list_low_intaracts= metric_list[:,np.array(n_intaracts)<10]
            results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list_low_intaracts), np.std(metric_list_low_intaracts)/np.sqrt(len(metric_list_low_intaracts))))
    results_all.append(results)
results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
print("All")
results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE-Stochastic,0.1475(0.0269),0.2343(0.0386),0.0165(0.0027),0.2953(0.0456),0.004123(0.000006)


In [53]:
# method_names = list(metrics_dic.keys())
# metric_names = list(metrics_dic[method_names[0]].keys())

# results_all = []
# for method_name, metrics in metrics_dic.items():
#     results = []
#     for metric_name, metric_list in metrics.items():
#         metric_list = np.array(metric_list)
#         if metric_name == "prediction_time":
#             results.append("{:6f}({:6f})".format(np.mean(metric_list), np.std(metric_list)/np.sqrt(len(metric_list))))
#         else:
#             metric_list_low_intaracts= metric_list[:,np.array(n_intaracts)<10]
#             results.append("{:4.4f}({:4.4f})".format(np.mean(metric_list_low_intaracts), np.std(metric_list_low_intaracts)/np.sqrt(len(metric_list_low_intaracts))))
#     results_all.append(results)
# results_all = pd.DataFrame(results_all, columns=metric_names, index=method_names)
# print("All")
# results_all

All


,ndcg@20,recall@20,precision@20,hit_rate@20,prediction_time
multi-VAE-Stochastic,0.1360(0.1010),0.2659(0.1804),0.0184(0.0124),0.3309(0.2104),0.004108(0.000031)
